<a href="https://colab.research.google.com/github/MutonyiLewis/DiabetesRecipes/blob/main/GINaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


##Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

##Preview the data


In [ ]:
gl_datapath = '/content/drive/MyDrive/Attachment Folder/FoodsWithGIs.csv'
gl_df = pd.read_csv(gl_datapath)
gl_df.head(10)

,Id,Food ID,Calories(kcal),Total Carbohydrate (g),Dietary Fiber (g),Sugars,Protein (g),Fat (g),Glucose,Mixed meal,Gl (%),GL,High Gl (%),Low GI (%),High GL
0,1,Corndog,210,26,2.0,7,5,10.0,2754 ± 762,1424 ± 401,53 ± 14,13 ± 3,67,39,16
1,2,Roasted mixed grain powde,125,23,55.0,3,8,1.3,2747 ± 1320,1372 ± 491,54 ± 18,9 ± 3,72,36,12
2,3,Chocolate cereal bar with nuts,246,25,4.0,14,8,14.0,2744 ± 442,1936 ± 272,72± 14,15 ± 3,86,58,18
3,4,Sweet potato salad,210,25,1.0,15,1,12.0,2877 ± 251,1574 ± 542,55 ± 18,13 ± 4,73,37,17
4,5,Pork and burdock root fried rice,175,26,2.0,3,11,3.0,3376 ± 160,1990 ± 129,60±24,15±6,84,36,21
5,6,Strawberry smoothie,100,25,2.0,22,1,0.0,2898 ± 972,2049 ± 684,67 ± 13,15 ±3,80,54,18
6,7,Crispy tofu in garlic teriyaki,230,26,6.0,10,18,7.0,3077 ± 858,499 ± 194,18±70,4 ± 1,88,-52,5
7,8,Sweet pumpkin and chestnut soup,180,25,0.0,15,4,7.0,3533 ± 791,1328 ± 399,37 ± 6,9±1,43,31,10
8,9,Lentils and Rice with braised tofu,340,50,5.0,6,19,8.0,4139+ 115,2583 ± 142,61 ± 21,27 ± 9,82,40,36
9,10,Spaghetti with tomato meat sauce and cheese,305,50,2.0,8,11,7.0,4075 ± 122,2044 ± 117,49 ± 19,24 ± 9,68,30,33


##Train test split

In [ ]:
# Load the training data
X = gl_df.drop(columns=['Id','Food ID','Sugars','Glucose','Mixed meal','Gl (%)', 'GL','High Gl (%)', 'Low GI (%)','High GL'])
y = gl_df['High Gl (%)']

##Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

##Define hyperparameters

In [ ]:
param_grid = {
    "var_smoothing": [1e-9, 1e-8, 1e-7, 1e-6, 1e-5],
    "priors": [None, [1, 1]]
}

##Create Model

In [ ]:
model = GaussianNB()

###Perform Grid search

In [ ]:
# Perform a grid search to find the optimal hyperparameters
grid_search = GridSearchCV(model, param_grid, cv=3)
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py", line 267, in fit
    return self._partial_fit(
  Fil

GridSearchCV(cv=3, estimator=GaussianNB(),
             param_grid={'priors': [None, [1, 1]],
                         'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05]})

##Get the best Model

In [ ]:
# Get the best model
best_model = grid_search.best_estimator_

##Evaluate the model

In [ ]:
# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
score = np.mean(np.square(y_test - y_pred))
print("Mean squared error on test set: ", score)

Mean squared error on test set:  82.875


##Use model to train new dataset

In [ ]:
recipes_dataset = pd.read_csv('/content/drive/MyDrive/Attachment Folder/Recipes.csv')
recipes_dataset.head(3)

df = pd.DataFrame(recipes_dataset)
df = df.drop(columns=['title','index','page','about','ingridients','preparation', 'nutrition per 100g of recipe','vitamin A(mcg)', 'iron(mg)','zinc(mg)'])
df['High Gl (%)'] = ''
y = df['High Gl (%)']
print(y)


0       
1       
2       
3       
4       
      ..
137     
138     
139     
140     
141     
Name: High Gl (%), Length: 142, dtype: object


In [ ]:
X1 = pd.DataFrame(df)
X1['Calories(kcal)'] = df['energy(kcal)']
X1['Total Carbohydrate (g)'] = df['carbohydrates(g)']
X1['Dietary Fiber (g)'] = df['fibre(g)']
X1['Protein (g)'] = df['proteins(g)']
X1['Fat (g)'] = df['fat(g)']
X1 = X1.drop(columns=['High Gl (%)','energy(kcal)', 'fibre(g)', 'fat(g)', 'proteins(g)', 'carbohydrates(g)'])
print(X1)
X1.head(3)

     Calories(kcal)  Total Carbohydrate (g)  Dietary Fiber (g)  Protein (g)  \
0             429.0                    52.8                1.6          4.6   
1             413.0                    46.6                2.1          6.0   
2             379.0                    49.9                2.2          7.6   
3             340.0                    48.7                2.1          6.4   
4             443.0                    40.5                3.1         18.8   
..              ...                     ...                ...          ...   
137           337.0                    35.9                1.7          5.9   
138            93.0                    14.5                1.8          1.1   
139           412.0                    39.4                1.1          4.3   
140            60.0                     8.0                0.0          2.0   
141           298.0                     9.6                3.5         10.6   

     Fat (g)  
0       21.8  
1       22.1  
2     

,Calories(kcal),Total Carbohydrate (g),Dietary Fiber (g),Protein (g),Fat (g)
0,429.0,52.8,1.6,4.6,21.8
1,413.0,46.6,2.1,6.0,22.1
2,379.0,49.9,2.2,7.6,16.1


In [ ]:
#Add the GL column
y_pred = best_model.predict(X1)
df['High Gl (%)'] = y_pred
print(y_pred)
df.head(45)

[80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80
 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80
 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80
 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80
 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80
 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80 80]


,energy(kcal),fat(g),carbohydrates(g),proteins(g),fibre(g),High Gl (%),Calories(kcal),Total Carbohydrate (g),Dietary Fiber (g),Protein (g),Fat (g)
0,429.0,21.8,52.8,4.6,1.6,80,429.0,52.8,1.6,4.6,21.8
1,413.0,22.1,46.6,6.0,2.1,80,413.0,46.6,2.1,6.0,22.1
2,379.0,16.1,49.9,7.6,2.2,80,379.0,49.9,2.2,7.6,16.1
3,340.0,12.9,48.7,6.4,2.1,80,340.0,48.7,2.1,6.4,12.9
4,443.0,22.2,40.5,18.8,3.1,80,443.0,40.5,3.1,18.8,22.2
5,325.0,13.1,40.2,8.5,6.2,80,325.0,40.2,6.2,8.5,13.1
6,270.0,8.5,39.2,8.3,1.7,80,270.0,39.2,1.7,8.3,8.5
7,449.0,29.8,37.4,7.1,1.4,80,449.0,37.4,1.4,7.1,29.8
8,241.0,5.6,41.2,5.1,2.8,80,241.0,41.2,2.8,5.1,5.6
9,0.0,0.0,0.0,0.0,0.0,80,0.0,0.0,0.0,0.0,0.0
